### Balanced CORD-NER dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab Notebooks/NER/Balance Dataset

/content/drive/MyDrive/Colab Notebooks/NER/Balance Dataset


In [ ]:
ls

balanced_dataset_ner.csv         sent_type_info.csv
corpus_for_ner_cleaned_ver3.csv  sent_type_info_union.csv
sent_type_count_add1.csv         test_balance_dataset_1.ipynb
sent_type_count.csv              test_balance_dataset_2.ipynb
sent_type_count_union.csv        test_balance_dataset_3.ipynb
sent_type_info_add1.csv


In [ ]:
import pandas as pd
import json
import time

start_time = time.time()

In [ ]:
df = pd.read_csv('corpus_for_ner_cleaned_ver3.csv')

In [ ]:
df['tag'] =  df['flag'].apply(lambda x: x[3:-1])

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,sent_number,sent_tokens,flag,tag
0,0,0,0,sentence 0,angiotensin_converting_enzyme,'B-GENE_OR_GENOME',GENE_OR_GENOME
1,1,1,1,sentence 0,2,'I-GENE_OR_GENOME',GENE_OR_GENOME
2,2,2,2,sentence 0,(,O,
3,3,3,3,sentence 0,ace2,'B-GENE_OR_GENOME',GENE_OR_GENOME
4,4,4,4,sentence 0,),O,


In [ ]:
# all possible 63 types

all_types = [
'ARCHAEON','PHYSICAL_SCIENCE','EDUCATIONAL_ACTIVITY','GROUP_ATTRIBUTE','MACHINE_ACTIVITY',
'LANGUAGE','INJURY_OR_POISONING','WORK_OF_ART','INDIVIDUAL_BEHAVIOR','EVENT',
'HUMAN-CAUSED_PHENOMENON_OR_PROCESS','BACTERIUM','GOVERNMENTAL_OR_REGULATORY_ACTIVITY','ANATOMICAL_STRUCTURE','MATERIAL',
'SOCIAL_BEHAVIOR','VIRAL_PROTEIN','PERCENT','DAILY_OR_RECREATIONAL_ACTIVITY','ORGAN_OR_TISSUE_FUNCTION',
'LOC','FAC','LAW','WILDLIFE','CELL_OR_MOLECULAR_DYSFUNCTION',
'MONEY','EXPERIMENTAL_MODEL_OF_DISEASE','FOOD','EVOLUTION','LABORATORY_OR_TEST_RESULT',
'DIAGNOSTIC_PROCEDURE','RESEARCH_ACTIVITY','PRODUCT','NORP','SUBSTRATE',
'CORONAVIRUS','CELL_FUNCTION','ORDINAL','BODY_SUBSTANCE','IMMUNE_RESPONSE',
'LIVESTOCK','TIME','SIGN_OR_SYMPTOM','VIRUS','MOLECULAR_FUNCTION',
'LABORATORY_PROCEDURE','EUKARYOTE','BODY_PART_ORGAN_OR_ORGAN_COMPONENT','THERAPEUTIC_OR_PREVENTIVE_PROCEDURE','QUANTITY',
'GPE','CELL_COMPONENT','TISSUE','ORGANISM','PERSON',
'CELL','GROUP','ORG','CARDINAL','DATE',
'DISEASE_OR_SYNDROME','CHEMICAL','GENE_OR_GENOME'
]

In [ ]:
# taken 25 types

taken_types = [
'INJURY_OR_POISONING','BACTERIUM','LOC','VIRUS','FAC',
'CORONAVIRUS','FOOD','LIVESTOCK','NORP','SUBSTRATE',
'IMMUNE_RESPONSE','BODY_SUBSTANCE','EUKARYOTE','SIGN_OR_SYMPTOM','BODY_PART_ORGAN_OR_ORGAN_COMPONENT',
'GPE','TISSUE','ORGANISM','CELL','ORG',
'GROUP','CARDINAL','DISEASE_OR_SYNDROME','GENE_OR_GENOME','CHEMICAL'
]

In [ ]:
# delete remaining types
del_types = list(set(all_types) - set(taken_types))

print(del_types[0:5])

['ARCHAEON', 'GROUP_ATTRIBUTE', 'CELL_FUNCTION', 'THERAPEUTIC_OR_PREVENTIVE_PROCEDURE', 'INDIVIDUAL_BEHAVIOR']


In [ ]:
# union smaller types' sentences and count all 25 
# if it is somehow balance, it is okay

In [ ]:
smaller_types = ['INJURY_OR_POISONING', 'BACTERIUM', 'FAC', 'FOOD']

In [ ]:
# union sent numbers and count 

union_sent_str = []
for entity_type in smaller_types:
  sent_str = df[df['tag'] == entity_type]['sent_number'].tolist()
  union_sent_str = list(set(union_sent_str + sent_str))

In [ ]:
print("# of sentences: ", len(union_sent_str))

# of sentences:  33211


In [ ]:
# try to add more sent # here 
# union_sent_str = sent col of selected min types

# add 3000 SIGN_OR_SYMPTOM
add_limit = 3000
add_types = ['SIGN_OR_SYMPTOM', 'LOC', 'BODY_SUBSTANCE', 'NORP', 
             'SUBSTRATE', 'IMMUNE_RESPONSE', 'CORONAVIRUS']

add_union_sent_str = []
for entity_type in add_types:
  sent_str = df[df['tag'] == entity_type]['sent_number'].tolist()[0:3000]
  add_union_sent_str = list(set(add_union_sent_str + sent_str))

In [ ]:
union_sent_str = list(set(add_union_sent_str + union_sent_str))

In [ ]:
print("# of sentences: ", len(union_sent_str))

# of sentences:  50557


In [ ]:
# sub_df of all the selected sentence #

sub_df = df[df['sent_number'].isin(union_sent_str)]

In [ ]:
#### Below part is to check for the statistical info of the sub_df

In [ ]:
sent_type_df = pd.DataFrame()

count_df = pd.DataFrame()

In [ ]:
for entity_type in taken_types:
  sent_str = list(set(sub_df[sub_df['tag'] == entity_type]['sent_number'].tolist()))
  sent_int = [int(sent_id[9:]) for sent_id in sent_str]
  sent_type_df[entity_type] = pd.Series(sent_int)

  count_df[entity_type] = pd.Series(len(sent_int))

In [ ]:
sent_type_df.head()

,INJURY_OR_POISONING,BACTERIUM,LOC,VIRUS,FAC,CORONAVIRUS,FOOD,LIVESTOCK,NORP,SUBSTRATE,...,GPE,TISSUE,ORGANISM,CELL,ORG,GROUP,CARDINAL,DISEASE_OR_SYNDROME,GENE_OR_GENOME,CHEMICAL
0,2005155,2501365,388192.0,1189983.0,908720,850467,1111624,2277795.0,2412020.0,2005155.0,...,1607405,850467.0,35109,132208,1111624,2091009,35109,27725,2501365,27725
1,1110951,2649007,103817.0,1271731.0,2091009,22521,2002672,2412020.0,166136.0,35109.0,...,1110951,1549746.0,908720,148110,908720,22521,2872086,1111624,148110,2501365
2,2277795,2344212,197865.0,81526.0,1549746,1347,2692661,81526.0,81526.0,196927.0,...,1090773,2798669.0,164936,1549746,164936,2798669,2344212,22521,164936,148110
3,2800312,157118,410787.0,769947.0,1139949,2405,2872086,2628209.0,303.0,2063911.0,...,234029,157118.0,2344212,1727982,1727982,186966,208054,1549746,769947,164936
4,641644,2510135,440070.0,1911550.0,1004502,2336395,2694283,1327668.0,127601.0,23976.0,...,2048614,2068623.0,769947,1472722,259462,23976,2694283,1765073,127601,1765073


In [ ]:
selected_df = sub_df[['sent_number', 'sent_tokens', 'flag']]

In [ ]:
selected_df['flag'] =  selected_df['flag'].apply(lambda x: 'del' if (x[3:-1] in del_types) else x)
selected_df.drop(selected_df[selected_df.flag == 'del'].index, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
selected_df.to_csv('balanced_dataset_ner.csv')

In [ ]:
sent_type_df.to_csv('sent_type_info_add1.csv')

count_df.to_csv('sent_type_count_add1.csv')

In [ ]:
end_time = time.time()

estimate_runtime = round((end_time-start_time)/60)
print('Estimate runtime: ', (end_time-start_time)/60, ' minutes.')

Estimate runtime:  2.168568825721741  minutes.


In [ ]:
# Estimate runtime:   2.2063471992810566  minutes.